<b><i>Цель исследования:</i></b> проанализивароть АБ тест, где тестируется новая формула ранжирования экскурсий на странице листинга города<p> 

<b>Оцениваем метрики:</b><p>
    конверсия в создание заказа<p>
    выручка

In [ ]:
#импортируем библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats as st
import random
import seaborn as sns
import statsmodels.stats.proportion as proportion


from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import norm
import matplotlib.pyplot as plt


from tqdm.auto import tqdm
plt.style.use('ggplot')

In [ ]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [ ]:
metrics_data = pd.read_csv('C:\\Users\\gatsko.veronika\\Desktop\\abt\\online_metrics.csv')#замените путь, чтобы код работал)
orders_data = pd.read_csv('C:\\Users\\gatsko.veronika\\Desktop\\abt\\orders.csv')# и туть

In [ ]:
metrics_data.head(5)

In [ ]:
metrics_data.info()

In [ ]:
sns.catplot(x="date", y="session_id", data=metrics_data)

<b>Конверсия</b>

Распределение данных в группах:

In [ ]:
metrics_data['split'].value_counts(normalize = True)

90% данных в группе rank_review<br>
10% группа original       

Исходя из того, что в задании описано, что в результате этого теста хочется увидеть рост конверсии в создание заказа, необходимо понять какое именно действие будем считать в этом случае.<br>
Заказ создается после клика на кнопку "Отправить" на странице бронирования.<br>
По описанию датасета нам нужно событие <b>cnt_booking_order_button</b><br><br>
Конверсию в создание заказа будем считать так:<br><br>
отношение количества кликнувших на кнопку "Отправить" на странице бронирования (cnt_booking_order_button) к количеству посещений страницы бронирования (cnt_booking) 

In [ ]:
result = metrics_data.groupby(['split']).agg({'cnt_booking':'sum', 'cnt_booking_order_button':'sum'})
result['cr'] = result['cnt_booking_order_button']/result['cnt_booking']*100
result

Для оценки того, отличаются ли результаты в тестируемых вариантах, какой вариант лучше и можно ли считать результаты значимыми, в данном случае можно воспользоваться одним из калькуляторов достоверности АB-тестирования, например вот этим:
https://mindbox.ru/academy/ab-test-calculator/
<br>
<b>Вывод:</b>
Варианты A и B значимо не различаются


<br>
<br>
<br>
<b>Выручка</b><br>

для меня неочвидно, возможно ли (если да, то как) имея предоставленны датафрейм оценить именно выручку.<br>
Предположим, что наша выручка это значения commission_rub - комиссии Tripster в рублях

Оценивать будем столбец <b>commission_rub</b><br>
наиболее подходящий из имеющийся

In [ ]:
orders_data['split'].value_counts(normalize = True)

90% данных в группе rank_review<br>
10% группа original    

In [ ]:
orders_data.groupby(['split']).agg({'order_id':'count'})

Видим, что количество наблюдений в группах отличается в разы. Однако, вопреки распространенному мнению, это не является проблемой.<br><br>Единственное, что для меня тут удивительно: обычно экспериментальная группа меньше, чем контрольная (если они разные), так как есть опасения давать тестовую версию большому количеству пользователей из-за возможных негативных последствий, а у вас наоборот она намного больше. Сможете рассказать почему?

In [ ]:
sns.boxplot(x="split", y="commission_rub", data=orders_data.query('is_cancelled == 0'))

При анализе не будем учитывать случаи с отмененными заказами, так как там всегда комиссия 0 и большое количество таких строк тянет распределение влево. Там прямо пик будет большой около нуля

Для выбора статистический критерия посмотрим распределение в группе <b>original</b>

In [ ]:
orders_data.query('split == "original"')['commission_rub'].hist()

а вот и пик около нуля

In [ ]:
orders_data.query('split == "original" & is_cancelled == 0')['commission_rub'].hist()

In [ ]:
orders_data.query('split == "original" & is_cancelled == 0').boxplot('commission_rub')

<b>Анализ диаграммы размаха:</b><br>
Нижний «ус» упирается в 0 - это минимальное значение.<br>
Верхний заканчивается около <b>8200</b> рублей — зто ориентировочная граница нормального размаха.<br>
Всё, что лежит за ней, считается выбросом.<br>
Более высокие значения изображены отдельными точками, но сливаются в единую жирную линию: таких выбросов очень много.

In [ ]:
len(orders_data.query('commission_rub == 0 & is_cancelled == 0 & split == "original"'))

Наблюдаем 27 случаев, где комиссиия 0, при неотмененном заказе. Имея только эти данные невозможно предположить от чего зависит размер комисии. Есть аномально высокие размеры комисиий, например 24806.4. Необходимо разобраться с аномалиями в данных

In [ ]:
o = orders_data.query('split == "original" & is_cancelled == 0 & commission_rub < 8200 & commission_rub != 0')['commission_rub']


In [ ]:
o.describe()

In [ ]:
len(orders_data.query('is_cancelled == 0 & split == "original" & commission_rub > 8200'))                                                                                                                      

146 случаев с аномально высокой комиссией. Попробуем отсечь эти значения.

In [ ]:
orders_data.query('split == "original" & is_cancelled == 0 & commission_rub < 8200 & commission_rub != 0').boxplot('commission_rub')


In [ ]:
orders_data.query('split == "original" & is_cancelled == 0 & commission_rub < 8200 & commission_rub != 0')['commission_rub'].hist()



Также посмотрим распределение в группе <b>rank_review</b>

In [ ]:
orders_data.query('split == "rank_review"')['commission_rub'].hist()

Отсекаем отмененные заказы

In [ ]:
orders_data.query('split == "rank_review" & is_cancelled == 0')['commission_rub'].hist()

In [ ]:
orders_data.query('split == "rank_review" & is_cancelled == 0').boxplot('commission_rub')

<b>Анализ диаграммы размаха:</b><br>
Нижний «ус» упирается в 0 - это минимальное значение.<br>
Верхний заканчивается около <b>9000</b> рублей — это ориентировочная граница нормального размаха.<br>
Всё, что лежит за ней, считается выбросом.<br>
Более высокие значения изображены отдельными точками, но сливаются в единую жирную линию: таких выбросов очень много как и в группе original.

In [ ]:
len(orders_data.query('commission_rub == 0 & is_cancelled == 0 & split == "rank_review"'))

Наблюдаем 192 случая, где комиссиия 0, при неотмененном заказе. Имея только эти данные невозможно предположить от чего зависит размер комисии. Есть аномально высокие размеры комисиий, например 56784. Необходимо разобраться с аномалиями в данных

In [ ]:
r = orders_data.query('split == "rank_review" & is_cancelled == 0 & commission_rub != 0')['commission_rub']


In [ ]:
r.describe()

In [ ]:
len(orders_data.query('is_cancelled == 0 & split == "rank_review" & commission_rub > 9000'))

Попробуем отсечь случаи с аномально высокими значениями

In [ ]:
orders_data.query('split == "rank_review" & is_cancelled == 0 & commission_rub < 9000 & commission_rub != 0').boxplot('commission_rub')

In [ ]:
orders_data.query('split == "rank_review" & is_cancelled == 0 & commission_rub < 9000 & commission_rub != 0')['commission_rub'].hist()

На гистограмме частот видно, что данные распределеные ненормально. Распределение скошено и имеет длинный хвост справа. Это означает, что есть выбросы - нетипичные, предельные значения. <br>
Как избоавляетесь от выбросов?<br>
допустим ожидаете нормальное распределение, а оно скошено и имеет длинный хвост справа?<br>
<br>
просто отсечь выбросы? <br>
или напрмер логарифмирование?<br>
или непараметрика

Нам нужно сравнить среднее двух выборок группы (А и Б). Как это сделать?<br>
У нас не так много опций.<br> Мы можем использовать:<br>
t-test критерий Стьюдента, который предполагает некоторые требования к данным (знакома с различными точками зрения на требования к данным: кто-то говорит, что критерий Стьюдента можно использовать только при нормальном распределении, кто-то говорит, что это не обязательное требование, что достаточно того, что дисперии выборок близки и количество наблюдений было достаточно большим)<br>
его непараметрический аналог - критерий Манна Уитни<br>
bootstrap

Нулевая гипотеза:<br>
Выручки групп между собой равны<br><br>
Альтернативная гипотеза:<br>
Выручка в группе B выше

In [ ]:
alpha =  .05 #уровень статистической значимости
#если p-value окажется меньше него - отвергнем нулевую гипотезу

results = st.ttest_ind(
    orders_data.query('split == "original" & is_cancelled == 0 & commission_rub < 8200 & commission_rub != 0').commission_rub, 
    orders_data.query('split == "rank_review" & is_cancelled == 0 & commission_rub < 9000 & commission_rub != 0').commission_rub
,equal_var = False) #выборки у нас разного размера, поэтому установим значение параметра False

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу. Выручка группы В выше")

else:
    print("Не получилось отвергнуть нулевую гипотезу. Выручки групп между собой равны")

Вывод:<br>
новая формула ранжирования экскурсий на странице листинга города не оказала ожидаемых влияний на конверсию.<br>
однако выручка (сумма комиссии в рублях) в тестовой группе выше, чем в контрольной
